### Data Preparation & Loading

In [35]:
# Initial imports
import numpy as np
import pandas as pd
from pathlib import Path

%matplotlib inline

In [36]:
# Set the random seed for reproducibility
from numpy.random import seed

seed(1)
from tensorflow import random

random.set_seed(2)

In [37]:
# Load the stocks data
df_stocks = pd.read_csv(
    Path("../Project_2_work/api_data.csv"),
    index_col="date",
    infer_datetime_format=True,
    parse_dates=True,
)
df_stocks.head()

,open,high,low,close,volume,trade_count,vwap,symbol
date,,,,,,,,
2019-04-22,89.69,90.470,89.58,89.60,970834,9479,90.000447,BUD
2019-04-23,88.56,89.790,88.50,89.25,755451,7907,89.242818,BUD
2019-04-24,89.02,89.130,87.82,88.21,1038366,11509,88.324096,BUD
2019-04-25,86.89,87.020,86.00,86.64,1341678,13124,86.467163,BUD
2019-04-26,87.38,87.625,86.83,87.58,920430,9005,87.292918,BUD


In [38]:
# Isolating the closing data using a pivot table
all_close_prices = df_stocks.pivot(values=["close"], columns= "symbol")
all_close_prices.head()

close                                                        \
symbol        BUD    CGC   CRON     DEO    JAZZ     SAM     SPY     STZ   
date                                                                      
2019-04-22  89.60  48.25  16.86  162.58  125.97  271.34  290.27  205.46   
2019-04-23  89.25  47.41  16.28  163.80  129.78  277.73  292.88  207.48   
2019-04-24  88.21  48.37  16.59  164.13  129.42  276.43  292.23  210.14   
2019-04-25  86.64  48.20  16.52  164.17  128.53  300.00  292.05  212.16   
2019-04-26  87.58  49.91  17.16  166.48  130.99  304.56  293.41  212.16   

                                 
symbol        TAP   TLRY    VFF  
date                             
2019-04-22  61.16  51.82  11.56  
2019-04-23  61.28  50.26  11.08  
2019-04-24  62.51  51.30  12.29  
2019-04-25  62.80  51.27  11.62  
2019-04-26  63.83  53.15  11.57

### Creating input feature vectors "X" and the target vector "y"

In [39]:
#The function will have the following parameters: df_stocks, window, feature_col_number, and target_col_number. 

def window_data(df_stocks, window, feature_col_number, target_col_number):

    X = []
    y = []
    for i in range(len(df_stocks) - window):
        features = df_stocks.iloc[i : (i + window), feature_col_number]
        target = df_stocks.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1,1)

In [52]:
window_size = 2

feature_column = [0,3,5,6,7,8]
target_column = 6
X, y = window_data(all_close_prices["close"], window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[[ 89.6  162.58 271.34 290.27 205.46  61.16]
  [ 89.25 163.8  277.73 292.88 207.48  61.28]]

 [[ 89.25 163.8  277.73 292.88 207.48  61.28]
  [ 88.21 164.13 276.43 292.23 210.14  62.51]]

 [[ 88.21 164.13 276.43 292.23 210.14  62.51]
  [ 86.64 164.17 300.   292.05 212.16  62.8 ]]

 [[ 86.64 164.17 300.   292.05 212.16  62.8 ]
  [ 87.58 166.48 304.56 293.41 212.16  63.83]]

 [[ 87.58 166.48 304.56 293.41 212.16  63.83]
  [ 88.01 166.14 304.09 293.87 212.54  63.88]]] 

y sample values:
[[292.23]
 [292.05]
 [293.41]
 [293.87]
 [294.02]]


### Splitting Data Between Training & Testing Sets 

In [41]:
# Using 90% of the data for training 
split = int(0.9 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

### Scaling the Data with MinMaxScaler

In [42]:
# Scaling the data before training the LSTM model using MinMaxScaler from sklearn to scale all values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#Fit scaler to reshaped X_Train
num_instances, num_time_steps, num_features = X_train.shape
X_train_reshaped = np.reshape(X_train, newshape=(-1, num_features))
scaler.fit(X_train_reshaped)
#Scale and reshape X_train to original shape
X_train_scaled = scaler.transform(X_train_reshaped)
X_train_scaled = np.reshape(X_train_scaled, newshape=(num_instances, num_time_steps, num_features))
#Scale and reshape X_test to original shape
num_instances, num_time_steps, num_features = X_test.shape
X_test_reshaped = np.reshape(X_test, newshape=(-1, num_features))
X_test_scaled = scaler.transform(X_test_reshaped)
X_test_scaled = np.reshape(X_test_scaled, newshape=(num_instances, num_time_steps, num_features))
#Fit scaler to y_train
scaler.fit(y_train)
#Scale y_train
y_train = scaler.transform(y_train)
#Scale y_test
y_test = scaler.transform(y_test)

### Build & Train the LSTM RNN/Defining the LSTM RNN Model Structure

In [43]:
# Build and Train LSTM RNN in Keeras and fit it using the training data defined above using Keras.
 # Import required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [44]:
# Define the LSTM RNN model by adding layers
# AND using the dropout layer to prevent overfitting. 
model = Sequential()

number_units = 100
dropout_fraction = 0.02 #Each epoch will randomly drop 20% of units to prevent overfitting.

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 6))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

### Compiling, Summarizing, and Training the Model

In [45]:
# Compiling model using adam optimizer and mean_square_error
model.compile(optimizer="adam", loss="mean_squared_error")

In [46]:
# Summarizing the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 2, 100)            42800     
                                                                 
 dropout_6 (Dropout)         (None, 2, 100)            0         
                                                                 
 lstm_7 (LSTM)               (None, 2, 100)            80400     
                                                                 
 dropout_7 (Dropout)         (None, 2, 100)            0         
                                                                 
 lstm_8 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                

In [47]:
# Since the model is now defined, it will be fit using 50 epochs. 
# Batch size will be set at 128. 
model.fit(X_train, y_train, epochs=50, shuffle=False, batch_size=128, verbose=1, validation_split=.1)

Epoch 1/50
5/5 [==============================] - 12s 517ms/step - loss: 0.1786 - val_loss: 0.3596
Epoch 2/50
5/5 [==============================] - 0s 32ms/step - loss: 0.0262 - val_loss: 0.1598
Epoch 3/50
5/5 [==============================] - 0s 26ms/step - loss: 0.0581 - val_loss: 0.2066
Epoch 4/50
5/5 [==============================] - 0s 26ms/step - loss: 0.0454 - val_loss: 0.3441
Epoch 5/50
5/5 [==============================] - 0s 23ms/step - loss: 0.0332 - val_loss: 0.3988
Epoch 6/50
5/5 [==============================] - 0s 26ms/step - loss: 0.0291 - val_loss: 0.3446
Epoch 7/50
5/5 [==============================] - 0s 24ms/step - loss: 0.0258 - val_loss: 0.2739
Epoch 8/50
5/5 [==============================] - 0s 25ms/step - loss: 0.0274 - val_loss: 0.2473
Epoch 9/50
5/5 [==============================] - 0s 24ms/step - loss: 0.0281 - val_loss: 0.2633
Epoch 10/50
5/5 [==============================] - 0s 25ms/step - loss: 0.0267 - val_loss: 0.2737
Epoch 11/50
5/5 [==========

### Model Performance

In [48]:
# Evaluating the model
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 3ms/step - loss: 0.0436


0.043638598173856735

In [49]:
# Make predictions
predicted = model.predict(X_test, batch_size=128)
predicted

1/1 [==============================] - 2s 2s/step


array([[0.72120506],
       [0.7202647 ],
       [0.70876426],
       [0.7001997 ],
       [0.67608595],
       [0.67545116],
       [0.69464153],
       [0.6853926 ],
       [0.6790281 ],
       [0.68732667],
       [0.6822882 ],
       [0.65543705],
       [0.64127886],
       [0.6367813 ],
       [0.62427074],
       [0.6030142 ],
       [0.592367  ],
       [0.5952925 ],
       [0.60329145],
       [0.6148301 ],
       [0.62730014],
       [0.6734499 ],
       [0.6988914 ],
       [0.7048976 ],
       [0.6886415 ],
       [0.68548244],
       [0.6940762 ],
       [0.686227  ],
       [0.68974024],
       [0.68697506],
       [0.6596429 ],
       [0.6481413 ],
       [0.69289285],
       [0.74339825],
       [0.7395443 ],
       [0.7295602 ],
       [0.718085  ],
       [0.6992585 ],
       [0.68885475],
       [0.70581526],
       [0.7073879 ],
       [0.73493725],
       [0.7460695 ],
       [0.7534423 ],
       [0.74806494],
       [0.74345917],
       [0.72700584],
       [0.712

### Plotting Predicted Vs. Real Prices

In [50]:
# Create a DataFrame of Real and Predicted values
df_values = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": predicted.ravel()
    }, index = df_stocks.index[-len(y_test): ])
df_values.head()

,Real,Predicted
date,,
2021-12-31,0.990099,0.721205
2022-01-03,1.000904,0.720265
2022-01-04,1.000275,0.708764
2022-01-05,0.964248,0.700200
2022-01-06,0.962519,0.676086


<AxesSubplot:xlabel='date'>

In [51]:
# PLotting real prices vs. predicted
df_values.plot(kind ="line")